In [1]:
! pip -qqq install autogen-agentchat~=0.2 matplotlib yfinance

In [2]:
import os
from typing import List

from IPython import get_ipython

from autogen import ConversableAgent
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor

In [3]:
class NotebookExecutor(CodeExecutor):

    @property
    def code_extractor(self) -> CodeExtractor:
        # Extact code from markdown blocks.
        return MarkdownCodeExtractor()

    def __init__(self) -> None:
        # Get the current IPython instance running in this notebook.
        self._ipython = get_ipython()

    def execute_code_blocks(self, code_blocks: List[CodeBlock]) -> CodeResult:
        log = ""
        for code_block in code_blocks:
            result = self._ipython.run_cell("%%capture --no-display cap\n" + code_block.code)
            log += self._ipython.ev("cap.stdout")
            log += self._ipython.ev("cap.stderr")
            if result.result is not None:
                log += str(result.result)
            exitcode = 0 if result.success else 1
            if result.error_before_exec is not None:
                log += f"\n{result.error_before_exec}"
                exitcode = 1
            if result.error_in_exec is not None:
                log += f"\n{result.error_in_exec}"
                exitcode = 1
            if exitcode != 0:
                break
        return CodeResult(exit_code=exitcode, output=log)

In [4]:
# Configure LLM using OllamaConfig
config_list = [{
    "model": "deepseek-r1:14b",
    "base_url": "http://localhost:11434",
    "api_type": "ollama"
}]

# Configure the assistant
llm_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.7,
}

In [5]:
code_writer_agent = ConversableAgent(
    name="CodeWriter",
    system_message="You are a helpful AI assistant.\n"
    "You use your coding skill to solve problems.\n"
    "You have access to a IPython kernel to execute Python code.\n"
    "You can suggest Python code in Markdown blocks, each block is a cell.\n"
    "The code blocks will be executed in the IPython kernel in the order you suggest them.\n"
    "All necessary libraries have already been installed.\n"
    "Once the task is done, returns 'TERMINATE'.",
    llm_config=llm_config,
)

code_executor_agent = ConversableAgent(
    name="CodeExecutor",
    human_input_mode = "NEVER",
    llm_config=False,
    code_execution_config={"executor": NotebookExecutor()},
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").strip().upper(),
)

In [7]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="Create a plot showing the market caps of the top 10 publicly listed companies using data from Yahoo Finance.",
)

CodeExecutor (to CodeWriter):

Create a plot showing the market caps of the top 10 publicly listed companies using data from Yahoo Finance.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CodeWriter (to CodeExecutor):

<think>
Alright, I need to create a plot showing the market caps of the top 10 publicly listed companies using data from Yahoo Finance. Let me break down how I can approach this.

First, I should figure out which libraries I'll need. Since I'm dealing with financial data, pandas and requests will be useful for handling and fetching data respectively. For plotting, matplotlib or seaborn would work well. I'll go with matplotlib because it's a fundamental library.

Next, I need to get the list of top companies. I can use Wikipedia since they have a readily available table of largest companies by market cap. The URL is something like 'https://en.wikipedia.org/wiki/List_of_largest_publicly_traded_companies'. Usin